In [12]:
import os
import re
import glob
from openai import OpenAI
from datetime import datetime
os.environ["DASHSCOPE_API_KEY"] = "sk-2ea9416b45e04af6b6aa72d3c2ade52f"

# 初始化OpenAI客户端
try:
    client = OpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
except Exception as e:
    print(f"初始化OpenAI客户端时出错: {e}")
    raise

# 定义输入和输出路径
product_categories_file = "./Data/产品类别.txt"
defect_analysis_pattern = "./生成结果/defect_analysis/*VOC数据_缺陷象限分析_完整列表.txt"
defect_prompt_file = "./Data/VOC-痛点-表格生成.md"
defect_output_dir = "./生成结果/defect_analysis/"

# 确保输出目录存在
os.makedirs(defect_output_dir, exist_ok=True)

# 读取产品类别文件
with open(product_categories_file, 'r', encoding='utf-8') as file:
    product_categories = file.read()

# 读取提示词文件
with open(defect_prompt_file, 'r', encoding='utf-8') as file:
    prompt_template = file.read()

# 获取所有匹配的缺陷分析文件
defect_analysis_files = glob.glob(defect_analysis_pattern)
print(f"找到 {len(defect_analysis_files)} 个匹配的文件")

processed_count = 0
error_count = 0
for defect_file in defect_analysis_files:
    # 提取文件名中的产品名称部分（*部分）
    file_basename = os.path.basename(defect_file)
    product_name = file_basename.split('VOC数据_缺陷象限分析_完整列表.txt')[0]
    
    # 构建输出文件名
    output_file = os.path.join(defect_output_dir, f"{product_name}VOC数据_缺陷象限表格汇总.md")
    
    # 读取缺陷分析文件
    with open(defect_file, 'r', encoding='utf-8') as file:
        defect_analysis = file.read()
    
    # 构建完整的提示内容
    prompt_content = f"""{prompt_template}

产品类别信息:
{product_categories}

缺陷分析数据:
{defect_analysis}
"""
    
    print(f"\n正在处理: {file_basename}")
    try:
        # 创建聊天完成请求
        completion = client.chat.completions.create(
            model="qwq-plus",
            messages=[
                {"role": "user", "content": prompt_content}
            ],
            stream=True
        )
        
        print("=" * 20 + "正在生成表格" + "=" * 20)
        answer_content = ""
        is_answering = False
        
        for chunk in completion:
            if not chunk.choices:
                continue
            
            delta = chunk.choices[0].delta
            
            # 收集回复内容
            if hasattr(delta, 'content') and delta.content is not None:
                if not is_answering:
                    print("\n开始接收模型输出...")
                    is_answering = True
                answer_content += delta.content
                print(".", end='', flush=True)  # 显示处理进度
        
        print("\n表格生成完成")
        
        # 检查是否包含markdown表格
        if "|" in answer_content and "-|-" in answer_content:
            print("检测到表格格式内容")
        else:
            print("警告: 未检测到表格格式，可能需要检查输出")
        
        # 为输出内容添加标题和时间戳
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        formatted_output = f"# {product_name} VOC数据缺陷象限分析表格汇总\n生成时间: {current_time}\n\n{answer_content}"
        
        # 将结果保存到输出文件
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(formatted_output)
        
        print("=" * 20 + "处理完成" + "=" * 20)
        print(f"结果已保存到 {output_file}")
        processed_count += 1
    
    except Exception as e:
        print(f"\n处理过程中出错: {e}")
        print(f"详细错误信息: {type(e).__name__}")
        error_count += 1
print(f"\n所有处理完成！成功: {processed_count} 个文件，失败: {error_count} 个文件")

找到 1 个匹配的文件

正在处理: 双竹2025-01-10之后VOC数据_缺陷象限分析_完整列表.txt
====================正在生成表格====================

开始接收模型输出...
.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [14]:
matches_analysis_pattern = "./生成结果/matches_analysis/*VOC数据_功能场景匹配象限分析_完整列表.txt"
matches_prompt_file = "./Data/VOC-场景匹配-表格生成.md"
matches_output_dir = "./生成结果/matches_analysis/"

# 确保输出目录存在
os.makedirs(matches_output_dir, exist_ok=True)

# 读取产品类别文件
with open(product_categories_file, 'r', encoding='utf-8') as file:
    product_categories = file.read()

# 读取提示词文件
with open(matches_prompt_file, 'r', encoding='utf-8') as file:
    prompt_template = file.read()

# 获取所有匹配的缺陷分析文件
matches_analysis_files = glob.glob(matches_analysis_pattern)
print(f"找到 {len(matches_analysis_files)} 个匹配的文件")

processed_count = 0
error_count = 0
for matches_file in matches_analysis_files:
    # 提取文件名中的产品名称部分（*部分）
    file_basename = os.path.basename(matches_file)
    product_name = file_basename.split('VOC数据_功能场景匹配象限分析_完整列表.txt')[0]
    
    # 构建输出文件名
    output_file = os.path.join(matches_output_dir, f"{product_name}VOC数据_功能场景匹配表格汇总.md")
    
    # 读取缺陷分析文件
    with open(matches_file, 'r', encoding='utf-8') as file:
        matches_analysis = file.read()
    
    # 构建完整的提示内容
    prompt_content = f"""{prompt_template}

产品类别信息:
{product_categories}

功能场景匹配数据:
{matches_analysis}

"""
    
    print(f"\n正在处理: {file_basename}")
    try:
        # 创建聊天完成请求
        completion = client.chat.completions.create(
            model="qwq-plus",
            messages=[
                {"role": "user", "content": prompt_content}
            ],
            stream=True
        )
        
        print("=" * 20 + "正在生成表格" + "=" * 20)
        answer_content = ""
        is_answering = False
        
        for chunk in completion:
            if not chunk.choices:
                continue
            
            delta = chunk.choices[0].delta
            
            # 收集回复内容
            if hasattr(delta, 'content') and delta.content is not None:
                if not is_answering:
                    print("\n开始接收模型输出...")
                    is_answering = True
                answer_content += delta.content
                print(".", end='', flush=True)  # 显示处理进度
        
        print("\n表格生成完成")
        
        # 检查是否包含markdown表格
        if "|" in answer_content and "-|-" in answer_content:
            print("检测到表格格式内容")
        else:
            print("警告: 未检测到表格格式，可能需要检查输出")
        
        # 为输出内容添加标题和时间戳
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        formatted_output = f"# {product_name} VOC数据缺陷象限分析表格汇总\n生成时间: {current_time}\n\n{answer_content}"
        
        # 将结果保存到输出文件
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(formatted_output)
        
        print("=" * 20 + "处理完成" + "=" * 20)
        print(f"结果已保存到 {output_file}")
        processed_count += 1
    
    except Exception as e:
        print(f"\n处理过程中出错: {e}")
        print(f"详细错误信息: {type(e).__name__}")
        error_count += 1

print(f"\n所有处理完成！成功: {processed_count} 个文件，失败: {error_count} 个文件")

找到 1 个匹配的文件

正在处理: 双竹2025-01-10之后VOC数据_功能场景匹配象限分析_完整列表.txt
====================正在生成表格====================

开始接收模型输出...
..................................................................................................................................................................................................................................................................................
表格生成完成
检测到表格格式内容
====================处理完成====================
结果已保存到 ./生成结果/matches_analysis/双竹2025-01-10之后VOC数据_功能场景匹配表格汇总.md

所有处理完成！成功: 1 个文件，失败: 0 个文件


In [16]:
needs_analysis_pattern = "./生成结果/needs_analysis/*VOC数据_用户需求象限分析_完整列表.txt"
needs_prompt_file = "./Data/VOC-用户需求-表格生成.md"
needs_output_dir = "./生成结果/needs_analysis/"

# 确保输出目录存在
os.makedirs(needs_output_dir, exist_ok=True)

# 读取产品类别文件
with open(product_categories_file, 'r', encoding='utf-8') as file:
    product_categories = file.read()

# 读取提示词文件
with open(needs_prompt_file, 'r', encoding='utf-8') as file:
    prompt_template = file.read()

# 获取所有匹配的缺陷分析文件
needs_analysis_files = glob.glob(needs_analysis_pattern)
print(f"找到 {len(needs_analysis_files)} 个匹配的文件")

processed_count = 0
error_count = 0
for needs_file in needs_analysis_files:
    # 提取文件名中的产品名称部分（*部分）
    file_basename = os.path.basename(needs_file)
    product_name = file_basename.split('VOC数据_用户需求象限分析_完整列表.txt')[0]
    
    # 构建输出文件名
    output_file = os.path.join(needs_output_dir, f"{product_name}VOC数据_用户需求匹配表格汇总.md")
    
    # 读取缺陷分析文件
    with open(needs_file, 'r', encoding='utf-8') as file:
        needs_analysis = file.read()
    
    # 构建完整的提示内容
    prompt_content = f"""{prompt_template}

产品类别信息:
{product_categories}

未被满足的需求数据:
{needs_analysis}

"""
    
    print(f"\n正在处理: {file_basename}")
    try:
        # 创建聊天完成请求
        completion = client.chat.completions.create(
            model="qwq-plus",
            messages=[
                {"role": "user", "content": prompt_content}
            ],
            stream=True
        )
        
        print("=" * 20 + "正在生成表格" + "=" * 20)
        answer_content = ""
        is_answering = False
        
        for chunk in completion:
            if not chunk.choices:
                continue
            
            delta = chunk.choices[0].delta
            
            # 收集回复内容
            if hasattr(delta, 'content') and delta.content is not None:
                if not is_answering:
                    print("\n开始接收模型输出...")
                    is_answering = True
                answer_content += delta.content
                print(".", end='', flush=True)  # 显示处理进度
        
        print("\n表格生成完成")
        
        # 检查是否包含markdown表格
        if "|" in answer_content and "-|-" in answer_content:
            print("检测到表格格式内容")
        else:
            print("警告: 未检测到表格格式，可能需要检查输出")
        
        # 为输出内容添加标题和时间戳
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        formatted_output = f"# {product_name} VOC数据缺陷象限分析表格汇总\n生成时间: {current_time}\n\n{answer_content}"
        
        # 将结果保存到输出文件
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(formatted_output)
        
        print("=" * 20 + "处理完成" + "=" * 20)
        print(f"结果已保存到 {output_file}")
        processed_count += 1
    
    except Exception as e:
        print(f"\n处理过程中出错: {e}")
        print(f"详细错误信息: {type(e).__name__}")
        error_count += 1

print(f"\n所有处理完成！成功: {processed_count} 个文件，失败: {error_count} 个文件")

找到 1 个匹配的文件

正在处理: 双竹2025-01-10之后VOC数据_用户需求象限分析_完整列表.txt
====================正在生成表格====================

开始接收模型输出...
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [17]:
import os
import re
import glob
import pandas as pd

# 定义输入路径模式 - 包含所有三个文件夹
md_folders = [
    "./生成结果/defect_analysis/",
    "./生成结果/matches_analysis/",
    "./生成结果/needs_analysis/"
]

# 存储处理成功和失败的文件计数
success_count = 0
error_count = 0

# 处理所有文件夹中的文件
for folder in md_folders:
    md_files_pattern = f"{folder}*表格汇总.md"
    # 获取当前文件夹中所有匹配的MD文件
    md_files = glob.glob(md_files_pattern)
    print(f"\n在 {folder} 中找到 {len(md_files)} 个匹配的Markdown文件")
    
    if not md_files:
        print(f"警告: 在 {folder} 中没有找到匹配的文件。请检查路径: {md_files_pattern}")
        continue
        
    for md_file in md_files:
        try:
            print(f"\n正在处理: {os.path.basename(md_file)}")
            # 构建输出Excel文件名（与MD文件名一致，只是扩展名不同）
            excel_file = md_file.replace('.md', '.xlsx')
            # 读取MD文件内容
            with open(md_file, 'r', encoding='utf-8') as file:
                md_content = file.read()
            # 使用正则表达式提取表格内容
            # 匹配表格头部和内容
            table_pattern = r'\|(.*)\|\n\|([-:|\s]+)\|\n((?:\|.*\|\n)+)'
            tables = re.findall(table_pattern, md_content)
            if not tables:
                print(f"在文件 {md_file} 中未找到有效的Markdown表格")
                error_count += 1
                continue
            # 处理找到的第一个表格
            headers = [header.strip() for header in tables[0][0].split('|') if header.strip()]
            rows_text = tables[0][2].strip().split('\n')
            # 解析表格行
            rows = []
            for row_text in rows_text:
                row_data = [cell.strip() for cell in row_text.split('|')[1:-1]]  # 去掉首尾的空元素
                if row_data:  # 确保行不为空
                    # 确保行数据与标题列数匹配
                    if len(row_data) < len(headers):
                        # 如果数据列数少于标题列数，用空字符串填充
                        row_data.extend([''] * (len(headers) - len(row_data)))
                    rows.append(row_data[:len(headers)])  # 只取与标题数量相同的列
            # 创建DataFrame
            df = pd.DataFrame(rows, columns=headers)
            # 保存为Excel文件
            df.to_excel(excel_file, index=False)
            print(f"已将表格转换为Excel并保存到: {excel_file}")
            success_count += 1
        except Exception as e:
            print(f"处理文件 {md_file} 时出错: {str(e)}")
            error_count += 1

print(f"\n处理完成! 成功: {success_count} 个文件, 失败: {error_count} 个文件")


在 ./生成结果/defect_analysis/ 中找到 1 个匹配的Markdown文件

正在处理: 双竹2025-01-10之后VOC数据_缺陷象限表格汇总.md
已将表格转换为Excel并保存到: ./生成结果/defect_analysis\双竹2025-01-10之后VOC数据_缺陷象限表格汇总.xlsx

在 ./生成结果/matches_analysis/ 中找到 1 个匹配的Markdown文件

正在处理: 双竹2025-01-10之后VOC数据_功能场景匹配表格汇总.md
已将表格转换为Excel并保存到: ./生成结果/matches_analysis\双竹2025-01-10之后VOC数据_功能场景匹配表格汇总.xlsx

在 ./生成结果/needs_analysis/ 中找到 1 个匹配的Markdown文件

正在处理: 双竹2025-01-10之后VOC数据_用户需求匹配表格汇总.md
已将表格转换为Excel并保存到: ./生成结果/needs_analysis\双竹2025-01-10之后VOC数据_用户需求匹配表格汇总.xlsx

处理完成! 成功: 3 个文件, 失败: 0 个文件
